[github](https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/1_command_line_reinforcement_learning/treasure_on_right.py)
[course](https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/2-1-general-rl/)

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
np.random.seed(2)

In [4]:
N_STATES = 6   #1维世界的宽度
ACTIONS = ['left', 'right']  #探索者的可用动作
EPSILON = 0.9  #贪婪度 greddy
ALPHA = 0.1    #学习率
GAMMA = 0.9    #奖励递减值
MAX_EPISODES = 13  #最大回合数
FRESH_TIME = 0.01  #移动间隔时间

## Q表
> 将所有q value行为值放入其中，更新q表也就是更新行为准则，其y轴是state（探索者位置），x轴是action（探索者行为）

In [5]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),   #q_table全0初始化
        columns = actions,
    )
    #print(table)
    return table
q_table = build_q_table(N_STATES, ACTIONS)
q_table

,left,right
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0


In [9]:
def choose_action(state, q_table):
    state_ations = q_table.iloc[state, :]  #选出这个state所有action
    if (np.random.uniform() > EPSILON) or (state_ations.all() == 0):  #根据epsilon，探索或者利用
        action_name = np.random.choice(ACTIONS)  #探索
    else:
        action_name = state_ations.argmax()    #利用
    return action_name

In [10]:
#作出行为，环境反馈。即上个state作出action，在下一个state_得到reward。
#只有到达最右端才会得到r=1，其余为0

def get_env_feedback(S, A):
    # this is how agent will interact with the environment
    if A == 'right':
        if S == N_STATES - 2:  #terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:
        R = 0
        if S == 0:
            S_ = S  #reach the wall
        else:
            S_ = S - 1
    return S_, R

In [11]:
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

## Q学习算法：

![image.png](https://raw.githubusercontent.com/seaside2mm/github-photos/master/q2.png)
![image.png](https://raw.githubusercontent.com/seaside2mm/github-photos/master/q3.png)
![image.png](https://raw.githubusercontent.com/seaside2mm/github-photos/master/q4.png)

### Steps:
```
初始化q表
Repeat (for each step of episode):
    while not terminated：
        根据q表，决定策略，探索还是利用，从当前state，决定action
        作出action，观测reward，state_
        更新q表， 新q值 = 原来q值 + 学习效率*（真实值q_target - 估计值q_predict）
        state移动到state_
        环境更新
 ```
 
### 要求
> 1. 能够自己编写下面算法。
> 2. 可以根据不同参数，比如学习率，奖励递减值等，探究一下收敛速度等等，并且作图表示。

In [14]:
def rl():
    q_table = build_q_table(N_STATES, ACTIONS)  #初始化q table
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0 #回合初始位置
        is_terminated = False
        update_env(S, episode, step_counter) #环境更新
        while not is_terminated:
            A = choose_action(S, q_table)  
            S_, R = get_env_feedback(S, A)   #实施行为得到环境反馈
            q_predict = q_table.loc[S, A]  #估算的状态行为值
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_,:].max()    #实际状态行为值（回合没有结束）
            else:
                q_target = R  #回合结束
                is_terminated = True
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  #q_table更新
            S = S_
            
            update_env(S, episode, step_counter+1)  
            
            step_counter += 1
        return q_table

In [15]:
if __name__ == '__main__':
    q_table = rl()
    print("\r\nQ-table:\n")
    print(q_table)

                                
Q-table:

   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.1
5   0.0    0.0
